# 0. Import libraries

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from pyStrint.strint import strInt
from pyStrint import preprocess as pp

# 1. Load files

In [2]:
inputDir = './tutorial/demo/'
inputDir = '/public/wanwang6/test/tutorial/demo'
outDir = f'{inputDir}/results/'
sc_exp = pd.read_csv(f'{inputDir}/SC_exp.tsv',sep = '\t',header=0,index_col=0)
sc_meta = pd.read_csv(f'{inputDir}/SC_meta.tsv',sep = '\t',header=0,index_col=0)
st_exp = pd.read_csv(f'{inputDir}/ST_exp.tsv',sep = '\t',header=0,index_col=0)
st_coord = pd.read_csv(f'{inputDir}/ST_coord.tsv',sep = '\t',header=0,index_col=0)
st_decon = pd.read_csv(f'{inputDir}/ST_decon.tsv',sep = '\t',header=0,index_col=0)

In [4]:
# Append the addition lrdf to the our lrdb
species  = 'Human'
lr_list = pd.read_csv(f'{inputDir}/lr_df.csv',sep=',',header=0,index_col=0)
lr_df = pp.load_lr_df(species = species)
lr_list.columns = lr_df.columns
lr_df = pd.concat((lr_df,lr_list),axis = 0)
lr_df.index = list(range(len(lr_df)))

# 2. Parameters

In [ ]:
species = 'Human'
st_tp = 'st'
alpha, beta, gamma, delta, eta = [2, 0.001, 0.001, 0.1, 0.0005]

num_per_spot = 6
repeat_penalty = 10

# max_rep for gradient descent, choose accordingly.
max_rep = 10

# 3. Get cell model

In [11]:
import smurf
operator = smurf.SMURF(n_features=15, estimate_only=True)
sc_distribution = operator.smurf_impute(sc_exp.T).T

Running SCEnd on 2131 cells and 3247 genes
normalizing data by library size...
preprocessing data...
number of iteration:  1 / 10
number of iteration:  2 / 10
number of iteration:  3 / 10
number of iteration:  4 / 10
number of iteration:  5 / 10
number of iteration:  6 / 10
number of iteration:  7 / 10
number of iteration:  8 / 10
number of iteration:  9 / 10
number of iteration:  10 / 10


OSError: Cannot save file into a non-existent directory: '/public/wanwang6/test/tutorial/demo/demo_results'

In [13]:
sc_distribution.to_csv(f'{outDir}SC_smurf.tsv',sep = '\t',header=True,index=True)

In [ ]:
# or you can load the precomputed results
sc_distribution = pd.read_csv(f'{outDir}SC_smurf.tsv',sep = '\t',header=0,index_col=0)

# 4. Preprocess

In [14]:
sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_distribution,
                                                    sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)

Data clean is done! Using 3244 shared genes .


# 5. Cell selection

In [15]:
obj_spex = strInt(save_path = outDir, st_adata = st_adata, weight = st_decon,
                    sc_ref = sc_ref, sc_adata = sc_adata, cell_type_key = 'celltype', lr_df = lr_df,
                    st_tp = 'st',species = species)
obj_spex.prep()
sc_agg_meta = obj_spex.select_cells(p = 0, mean_num_per_spot = num_per_spot, mode = 'strict', max_rep = 1,
                        repeat_penalty = repeat_penalty)

sc_ref and sc_adata has different genes, both data are subset to 3244 genes.
Parameters checked!
Getting svg genes
By setting k as 6, each spot has average 5.80 neighbors.
	 Estimating the cell number in each spot by the deconvolution result.
	 SpexMod selects 3070 feature genes.
	 Init solution: max - 0.9984,     mean - 0.9801,     min - 0.8617
	 Swap selection start...
	 Swapped solution: max - 1.00,     mean - 0.99,     min - 0.89


# 6. Refinement process

In [16]:
# sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)
alter_sc_exp, sc_agg_meta = obj_spex.gradient_descent(
                alpha, beta, gamma, delta, eta,
                init_sc_embed = False,
                iteration = 10, k = 2, W_HVG = 2,
                left_range = 7, right_range = 8, steps = 1, dim = 2)

Running v12 now...
Init sc_coord by affinity embedding...
Avearge shape correlation is: 0.8778290098402767
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calculation done!
Hyperparameters adjusted.
-----Start iteration 0 -----
Avearge shape correlation is: 0.8778290098402767
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calculation done!
---0 self.loss4 939.0265198205159 self.GAMMA 3.2514940128800084e-06 self.GAMMA*self.loss4 0.003053239107131958
---In iteration 0, the loss is:loss1:3.05324,loss2:17.78077,loss3:4892.74138,loss4:939.02652,loss5:216.12637.
---In iteration 0, the loss is:loss1:3.05324,loss2:6.10648,loss3:0.00305,loss4:0.00305,loss5:0.30532.
The total loss after iteration 0 is 9.47115.
-----Start iteration 1 -----
Avearge shape correlation is: 0.8777965634613212
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calcula